In [ ]:
import pandas as pd

base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
base.shape

(21613, 21)

In [3]:
X = base.iloc[:, 5:6].values
y = base.iloc[:, 2:3].values

In [4]:
X

array([[1180],
       [2570],
       [ 770],
       ...,
       [1020],
       [1600],
       [1020]])

In [5]:
y

array([[221900.],
       [538000.],
       [180000.],
       ...,
       [402101.],
       [400000.],
       [325000.]])

In [6]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

In [7]:
X

array([[-0.97983502],
       [ 0.53363434],
       [-1.42625404],
       ...,
       [-1.15404732],
       [-0.52252773],
       [-1.15404732]])

In [8]:
y

array([[-0.86671733],
       [-0.00568792],
       [-0.98084935],
       ...,
       [-0.37586519],
       [-0.38158814],
       [-0.58588173]])

In [9]:
import tensorflow as tf
tf.__version__

2025-04-05 13:18:38.644485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 13:18:38.671936: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-05 13:18:38.671963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-05 13:18:38.672647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 13:18:38.677278: I tensorflow/core/platform/cpu_feature_guar

'2.15.0'

In [10]:
colunas =[tf.feature_column.numeric_column('previsor', shape=[1])]

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [11]:
colunas

[NumericColumn(key='previsor', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [12]:
config = tf.estimator.RunConfig(tf_random_seed=42)

Instructions for updating:
Use tf.keras instead.


In [14]:
regressor = tf.estimator.LinearRegressor(feature_columns=colunas, config=config)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpl88hk934', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
from sklearn.model_selection import train_test_split

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
X_treinamento.shape

(15129, 1)

In [17]:
y_treinamento.shape

(15129, 1)

In [18]:
X_teste.shape

(6484, 1)

In [19]:
y_teste.shape

(6484, 1)

In [24]:
def make_input_fn(X, y=None, num_epochs=None, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices(({'previsor': X}, y))
        if shuffle:
            ds = ds.shuffle(1_000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [25]:
funcao_treinamento = make_input_fn(X_treinamento, y_treinamento)

In [26]:
funcao_teste = make_input_fn(X_teste, y_teste, num_epochs=1_000, shuffle=False)

In [27]:
regressor.train(input_fn=funcao_treinamento, steps=10_0000)

INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2025-04-05 13:26:26.424622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-05 13:26:26.430491: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-04-05 13:26:26.535855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpl88hk934/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:loss = 0.8920282, step = 0
INFO:tensorflow:global_step/sec: 2317.99
INFO:tensorflow:loss = 0.30374563, step = 100 (0.044 sec)
INFO:tensorflow:global_step/sec: 2602.38
INFO:tensorflow:loss = 0.46357384, step = 200 (0.039 sec)
INFO:tensorflow:global_step/sec: 3002.77
INFO:tensorflow:loss = 0.241681, step = 300 (0.033 sec)
INFO:tensorflow:global_step/sec: 3594.74
INFO:tensorflow:loss = 0.37792256, step = 400 (0.027 sec)
INFO:tensorflow:global_step/sec: 2154.65
INFO:tensorflow:loss = 1.0386455, step = 500 (0.047 sec)
INFO:tensorflow:global_step/sec: 1540.68
INFO:tensorflow:loss = 0.20953715, step = 600 (0.064 

In [29]:
metricas_teste = regressor.evaluate(input_fn = funcao_teste, steps = 10000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2025-04-05T14:05:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpl88hk934/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2025-04-05 14:05:01.442937: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-05 14:05:01.448588: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 4.11889s
INFO:tensorflow:Finished evaluation at 2025-04-05-14:05:05
INFO:tensorflow:Saving dict for global step 100000: average_loss = 0.55276674, global_step = 100000, label/mean = 0.015570629, loss = 0.55620265, prediction/mean = 0.011391983
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: /tmp/tmpl88hk934/model.ckpt-100000


In [30]:
metricas_treinamento = regressor.evaluate(input_fn=funcao_treinamento, steps=10_000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2025-04-05T14:13:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpl88hk934/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2025-04-05 14:13:46.871848: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-05 14:13:46.876105: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 4.45997s
INFO:tensorflow:Finished evaluation at 2025-04-05-14:13:51
INFO:tensorflow:Saving dict for global step 100000: average_loss = 0.4879404, global_step = 100000, label/mean = -0.0065834196, loss = 0.48791012, prediction/mean = -0.0045910296
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: /tmp/tmpl88hk934/model.ckpt-100000


In [31]:
metricas_teste

{'average_loss': 0.55276674,
 'label/mean': 0.015570629,
 'loss': 0.55620265,
 'prediction/mean': 0.011391983,
 'global_step': 100000}

In [32]:
metricas_treinamento

{'average_loss': 0.4879404,
 'label/mean': -0.0065834196,
 'loss': 0.48791012,
 'prediction/mean': -0.0045910296,
 'global_step': 100000}

In [33]:
import numpy as np
novas_casas = np.array([[800], [900], [1_000]])
novas_casas

array([[ 800],
       [ 900],
       [1000]])

In [34]:
novas_casas = scaler_x.transform(novas_casas)
novas_casas

array([[-1.39358923],
       [-1.28470655],
       [-1.17582386]])

In [35]:
funcao_previsao = make_input_fn(novas_casas, num_epochs=1, shuffle=False)

In [36]:
previsoes = regressor.predict(input_fn=funcao_previsao)

In [37]:
previsoes

<generator object Estimator.predict at 0x754184445510>

In [38]:
list(previsoes)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpl88hk934/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2025-04-05 14:20:15.016073: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-05 14:20:15.019832: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[{'predictions': array([-0.97847384], dtype=float32)},
 {'predictions': array([-0.9020098], dtype=float32)},
 {'predictions': array([-0.8255456], dtype=float32)}]

In [44]:
for p in regressor.predict(input_fn=funcao_previsao):
    print(p['predictions'], scaler_y.inverse_transform([p['predictions']])[0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpl88hk934/model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[-0.97847384] [180872.09374172]
[-0.9020098] [208943.47917606]
[-0.8255456] [237014.90837435]


2025-04-05 14:23:27.561475: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-05 14:23:27.566671: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
